In [34]:
import pandas as pd
import numpy as np
import requests
from pytesseract import pytesseract
from PIL import Image as img
from urllib.request import Request, urlopen,urlretrieve
import copy

In [32]:
pytesseract.tesseract_cmd = r'c:\users\roshr\appdata\local\programs\python\python310\lib\site-packages'

In [26]:
df = pd.read_json('subspace.json')
df

,expiry_image,expiring_at,expiring_at_from_model,created_at,whatsub_plan
0,https://cdn.subspace.money/w_subscriptions/Xk5...,2023-05-13,2023-05-13,2022-05-15 04:27:17.955815+00:00,"{'duration': 3, 'duration_type': 'months'}"
1,https://cdn.subspace.money/w_subscriptions/PKV...,2023-03-29,None,2022-06-10 11:28:46.053421+00:00,"{'duration': 1, 'duration_type': 'years'}"
2,https://cdn.subspace.money/w_subscriptions/80E...,2022-12-27,2022-12-27,2022-07-21 16:30:53.316893+00:00,"{'duration': 1, 'duration_type': 'months'}"
3,https://cdn.subspace.money/w_subscriptions/JIX...,2022-10-22,2022-08-22,2022-07-22 15:02:53.685566+00:00,"{'duration': 1, 'duration_type': 'months'}"
4,https://cdn.subspace.money/w_subscriptions/p4r...,2022-07-12,None,2022-06-12 05:16:00.847964+00:00,"{'duration': 1, 'duration_type': 'months'}"
...,...,...,...,...,...
6723,https://cdn.subspace.money/w_subscriptions/OYl...,2023-10-07,2023-10-07,2022-10-07 16:03:36.596915+00:00,"{'duration': 12, 'duration_type': 'months'}"
6724,https://cdn.subspace.money/w_subscriptions/5av...,2022-11-07,None,2022-10-07 16:06:06.037587+00:00,"{'duration': 1, 'duration_type': 'months'}"
6725,https://cdn.subspace.money/w_subscriptions/X4e...,2022-12-31,None,2022-10-07 17:25:59.667282+00:00,"{'duration': 1, 'duration_type': 'months'}"
6726,https://cdn.subspace.money/w_subscriptions/qmT...,2022-11-07,None,2022-06-14 08:22:25.590519+00:00,"{'duration': 1, 'duration_type': 'months'}"


Accessing All Images

In [44]:
for i in df.iloc[:,0]:
    r = requests.get(i)
    print(i)
    with open('test.jpeg', 'wb') as out:
        out.write(r.content)
    im = Image.open("test.jpeg")
im.show()

https://cdn.subspace.money/w_subscriptions/Xk5bMvR8J7I_H6ywBPgSe.jpeg
https://cdn.subspace.money/w_subscriptions/PKVCqar0VD_Bd5ZeH5u3b.jpeg
https://cdn.subspace.money/w_subscriptions/80EDUPk1OPF2HsfPc2DxR.jpeg
https://cdn.subspace.money/w_subscriptions/JIXqhr0Z3tgejBXiHMe5G.jpeg
https://cdn.subspace.money/w_subscriptions/p4rpSF3m2Tk7MvYJes-rU.jpeg
https://cdn.subspace.money/w_subscriptions/hTm_kemQ5eXLY-hAG7g6a.jpeg
https://cdn.subspace.money/w_subscriptions/eCgajW4ozJSYuaedzwx8Z.jpeg
https://cdn.subspace.money/w_subscriptions/KHPHJ3vRAwyIpkTeL6od_.jpeg
https://cdn.subspace.money/w_subscriptions/ZjDa2LhXpqAqImRrYE6mX.jpeg
https://cdn.subspace.money/w_subscriptions/dTXEOgLfz4O8gHoSz8Ucz.jpeg
https://cdn.subspace.money/w_subscriptions/AQ8o5L1zpUnaMvxRFRifH.jpeg
https://cdn.subspace.money/w_subscriptions/BpkaedJRaSjHY_WPuvGyo.jpeg
https://cdn.subspace.money/w_subscriptions/F_xsSXo_xcRv8v84STSOW.jpeg
https://cdn.subspace.money/w_subscriptions/WaVyKgxn_yUresyhv2BKa.jpeg
https://cdn.subspace